## Importing Libraries

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Create Torch Dataset

In [23]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, 'r') as f:
            self.data = json.load(f)
        
        self.tranforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.tranforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)
        kps[::2] *= 224.0 / w
        kps[1::2] *= 224.0 / h
        
        return img, kps

In [18]:
train_dataset = KeypointsDataset("data/images", "data/data_train.json")
val_dataset = KeypointsDataset("data/images", "data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

## Create Model

In [19]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2)

In [20]:
model = model.to(device)

## Train Model

In [21]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [22]:
epochs = 20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()
        
        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14862.7802734375
Epoch 0, iter 10, loss: 15200.0498046875
Epoch 0, iter 20, loss: 14523.8076171875
Epoch 0, iter 30, loss: 14455.8193359375
Epoch 0, iter 40, loss: 13268.060546875
Epoch 0, iter 50, loss: 13373.724609375
Epoch 0, iter 60, loss: 12445.283203125
Epoch 0, iter 70, loss: 12095.6728515625
Epoch 0, iter 80, loss: 11882.748046875
Epoch 0, iter 90, loss: 11872.232421875
Epoch 0, iter 100, loss: 10811.9736328125
Epoch 0, iter 110, loss: 11049.474609375
Epoch 0, iter 120, loss: 10670.6103515625
Epoch 0, iter 130, loss: 10388.9072265625
Epoch 0, iter 140, loss: 9685.482421875
Epoch 0, iter 150, loss: 9029.78125
Epoch 0, iter 160, loss: 9352.494140625
Epoch 0, iter 170, loss: 8739.8505859375
Epoch 0, iter 180, loss: 8137.1884765625
Epoch 0, iter 190, loss: 7847.0341796875
Epoch 0, iter 200, loss: 7686.98486328125
Epoch 0, iter 210, loss: 7708.6865234375
Epoch 0, iter 220, loss: 7037.11083984375
Epoch 0, iter 230, loss: 6656.9951171875
Epoch 0, iter 240, loss:

In [24]:
torch.save(model.state_dict(), "keypoints_model.pth")